# Data Preprocessing for Image
This notebook contains code for preprocessing the raw images of guava fruits, including techniques such as unsharp masking, CLAHE, data augmentation, and normalization.

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Define paths
raw_data_dir = '../data/raw'
processed_data_dir = '../data/processed'
categories = ['anthracnose', 'fruit_flies', 'healthy']

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (512, 512))
    return image

# Function to apply unsharp masking
def unsharp_mask(image, sigma=1.0, strength=1.5):
    blurred = cv2.GaussianBlur(image, (0, 0), sigma)
    sharpened = cv2.addWeighted(image, 1 + strength, blurred, -strength, 0)
    return sharpened

# Function to apply CLAHE
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_Lab2RGB)

# Preprocess and save images
for category in categories:
    category_path = os.path.join(raw_data_dir, category)
    processed_category_path = os.path.join(processed_data_dir, category)
    os.makedirs(processed_category_path, exist_ok=True)
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        image = preprocess_image(img_path)
        image = unsharp_mask(image)
        image = apply_clahe(image)
        processed_img_path = os.path.join(processed_category_path, img_name)
        cv2.imwrite(processed_img_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Example of augmenting images
sample_image = preprocess_image(os.path.join(processed_data_dir, 'healthy', os.listdir(os.path.join(processed_data_dir, 'healthy'))[0]))
sample_image = np.expand_dims(sample_image, axis=0)
i = 0
for batch in datagen.flow(sample_image, batch_size=1):
    plt.imshow(batch[0].astype('uint8'))
    plt.axis('off')
    plt.show()
    i += 1
    if i > 5:
        break


ModuleNotFoundError: No module named 'keras'